## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-07-22-50-59 +0000,nyt,Trump Nominates Stephen Miran to Temporarily J...,https://www.nytimes.com/2025/08/07/us/politics...
1,2025-08-07-22-50-48 +0000,nypost,Witness describes panic to try to try to save ...,https://nypost.com/2025/08/07/us-news/witness-...
2,2025-08-07-22-48-45 +0000,nypost,Trump admin announces $50M reward for capture ...,https://nypost.com/2025/08/07/us-news/trump-ad...
3,2025-08-07-22-48-28 +0000,nyt,"Trump 401(k) Order Opens Door for Crypto, Priv...",https://www.nytimes.com/2025/08/07/business/tr...
4,2025-08-07-22-48-02 +0000,nypost,Tequila-toting nitwit gets stuck in California...,https://nypost.com/2025/08/07/us-news/tequila-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2305/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,77
326,tariffs,22
248,new,19
187,will,16
254,up,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
34,2025-08-07-21-50-00 +0000,wsj,President Trump said he would nominate Stephen...,https://www.wsj.com/politics/policy/stephen-mi...,134
95,2025-08-07-20-10-22 +0000,wapo,Trump celebrates new tariffs as negotiations c...,https://www.washingtonpost.com/business/2025/0...,133
116,2025-08-07-19-02-30 +0000,wapo,Here’s where the money from Trump’s tariffs wi...,https://www.washingtonpost.com/business/2025/0...,132
232,2025-08-07-11-54-12 +0000,bbc,Trump's sweeping new tariffs take effect again...,https://www.bbc.com/news/articles/cx23jmvn5yzo...,132
197,2025-08-07-14-30-28 +0000,wapo,Trump’s new tariffs are hitting these major tr...,https://www.washingtonpost.com/business/2025/0...,130


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
34,134,2025-08-07-21-50-00 +0000,wsj,President Trump said he would nominate Stephen...,https://www.wsj.com/politics/policy/stephen-mi...
297,79,2025-08-07-04-12-00 +0000,wsj,"China’s exports grew at a faster clip in July,...",https://www.wsj.com/economy/trade/chinas-expor...
79,77,2025-08-07-20-44-57 +0000,nypost,New Yorkers latching on to new breast milk-fla...,https://nypost.com/2025/08/07/us-news/new-york...
126,69,2025-08-07-18-30-00 +0000,wsj,Sen. John Cornyn said the FBI will help track ...,https://www.wsj.com/politics/policy/fbi-texas-...
141,39,2025-08-07-18-00-11 +0000,bbc,"Dozens of migrants detained under 'one in, one...",https://www.bbc.com/news/articles/ce35v0zyzvlo...
101,39,2025-08-07-20-00-25 +0000,bbc,Netanyahu says Israel intends to take complete...,https://www.bbc.com/news/articles/c8ryekj1m1do...
205,39,2025-08-07-13-55-44 +0000,nypost,Trump demands Intel CEO Lip-Bu Tan ‘resign imm...,https://nypost.com/2025/08/07/business/trump-d...
146,39,2025-08-07-17-51-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...
42,35,2025-08-07-21-45-20 +0000,nypost,Two dozen neglected dogs rescued from stomach-...,https://nypost.com/2025/08/07/us-news/two-doze...
225,33,2025-08-07-12-20-31 +0000,bbc,Why Trump-Putin talks unlikely to bring rapid ...,https://www.bbc.com/news/articles/c14gkkzvpx8o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
